# 🩺 Ecocardiografia à Beira do Leito (POCUS) na UTI

A ecocardiografia point-of-care é uma extensão do exame físico do intensivista. Neste notebook vamos revisar protocolos rápidos, interpretações de parâmetros e construir pequenas ferramentas para apoiar a tomada de decisão em tempo real.

## 🎯 Objetivos de Aprendizado
- Revisar a sistemática de aquisição de janelas ecocardiográficas na UTI.
- Reconhecer padrões à beira-leito associados a choque, congestão e ventilação mecânica.
- Utilizar medidas simples (VTI, VCI, E/e') para inferir perfusão e responsividade a fluidos.
- Integrar achados do POCUS com protocolos de ressuscitação e ventilação.

## 🗺️ Roteiro de Avaliação Rápida
```mermaid
graph TD
    A[Paciente instável] --> B{Via aérea segura?}
    B -->|Não| B1[Corrigir via aérea]
    B -->|Sim| C[Eco subxifoide]
    C --> D{Tamponamento?}
    D -->|Sim| D1[Pericardiocentese urgente]
    D -->|Não| E[Eco paraesternal eixo longo]
    E --> F{Disfunção ventricular grave?}
    F -->|Sim| F1[Inotrópicos + suporte avançado]
    F -->|Não| G[VCI + Doppler venoso]
    G --> H{Colapso > 50%?}
    H -->|Sim| H1[Reanimação volêmica]
    H -->|Não| I[Reavaliar ventilação / vasopressores]
```

## 🗒️ Checklist de Aquisição
1. **Preparação**: ajuste ganho, profundidade e configurações do transdutor.
2. **Eixo Paraesternal Longo**: avalie função ventricular global, derrame pericárdico e valvas AV.
3. **Eixo Paraesternal Curto**: inspeção de fração de encurtamento e movimento regional.
4. **Apical 4 câmaras**: estimativa de E/e' e avaliação de ventrículo direito.
5. **Subxifoide**: visibiliza derrame pericárdico, trombos cavitários e avaliação de VCI.
6. **Pulmonar**: sonda linear em hemitórax anterior para linhas A/B.

> ✅ **Dica**: Registre loops de 3 a 4 ciclos para análise e comparação seriada.

In [ ]:
import pandas as pd

protocolos = pd.DataFrame([
    ('Paraesternal Longo', '1. Mitral e aórtica', 'Fracionamento do VE, derrame'),
    ('Paraesternal Curto', '2. Eixo papilar', 'Movimento regional e FE qualitativa'),
    ('Apical 4 Câmaras', '3. Septo lateral', "E/e', função VD"),
    ('Subxifoide', '4. VE/VD + VCI', 'Tamponamento, colapso da VCI'),
    ('Pulmonar Anterior', '5. Intercostais', 'Perfil A/B, pneumotórax'),
], columns=['Janela', 'Sequência', 'Pergunta-chave'])

protocolos

### Como usar a tabela
- **Janela**: ordem sugerida para o intensivista solitário.
- **Sequência**: lembrete visual rápido para posicionamento do transdutor.
- **Pergunta-chave**: foco clínico para diferenciar diagnósticos imediatos.

Reaplicar a sequência a cada intervenção relevante (fluido, vasopressor, ajuste ventilatório).

In [ ]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider

pi = np.pi


def hemodynamics(vti_cm: float = 18.0, lvot_diam_mm: float = 20.0, heart_rate: float = 90.0):
    'Calcula volume sistólico, débito cardíaco e sugere interpretação clínica.'
    area = pi * (lvot_diam_mm / 10 / 2) ** 2  # área em cm²
    stroke_volume = area * vti_cm  # volume sistólico (mL)
    cardiac_output = stroke_volume * heart_rate / 1000  # L/min

    if cardiac_output < 3.5:
        status = '⚠️ Baixo débito: combine com avaliação clínica para considerar inotrópicos.'
    elif cardiac_output > 6.5:
        status = '🔺 Alto débito: investigar febre, anemia ou choque distributivo.'
    else:
        status = '✅ Débito dentro da faixa esperada (3.5–6.5 L/min).'

    fig = go.Figure(go.Indicator(
        mode='gauge+number',
        value=cardiac_output,
        number={'suffix': ' L/min'},
        gauge={
            'axis': {'range': [0, 10]},
            'steps': [
                {'range': [0, 3.5], 'color': '#f6d6d6'},
                {'range': [3.5, 6.5], 'color': '#d6f6e1'},
                {'range': [6.5, 10], 'color': '#ffe0b2'},
            ],
            'threshold': {'line': {'color': '#ef6c00', 'width': 4}, 'value': cardiac_output},
        },
        title={'text': 'Débito Cardíaco estimado'},
    ))

    fig.update_layout(height=300)
    fig.show()

    return {
        'Área LVOT (cm²)': round(area, 2),
        'Volume sistólico (mL)': round(stroke_volume, 1),
        'Débito cardíaco (L/min)': round(cardiac_output, 2),
        'Interpretação': status,
    }


interact(
    hemodynamics,
    vti_cm=FloatSlider(value=18, min=10, max=30, step=0.5, description='VTI (cm)'),
    lvot_diam_mm=FloatSlider(value=20, min=14, max=26, step=0.5, description='LVOT (mm)'),
    heart_rate=FloatSlider(value=90, min=40, max=150, step=5, description='FC (bpm)'),
)

## 📊 Avaliação da Congestão Venosa
- **VCI < 1,5 cm com colapso > 50%**: indica hipovolemia provável e resposta a fluido.
- **VCI > 2,1 cm sem colapso**: sugere pressão atrial direita elevada.
- **Doppler hepático pulsátil ou reverso**: corrobora congestão sistêmica.
- **Veia porta com fluxo alternante**: sensível a congestão significativa.

Integre com avaliação de B-lines (pulmonares) e ponto de ebulição hepática.

In [ ]:
from ipywidgets import interact_manual, FloatSlider


def avaliar_congestao(vci_diam_mm: float, colapso_percentual: float, doppler_hepatico: str):
    'Sugere categoria de congestionamento à beira do leito.'
    if vci_diam_mm < 15 and colapso_percentual > 50:
        categoria = 'Provável hipovolemia'
        conduta = 'Considerar desafio volêmico guiado por hemodinâmica e perfusão.'
    elif vci_diam_mm > 21 and colapso_percentual < 20:
        if doppler_hepatico == 'Reverso/alternante':
            categoria = 'Congestão grave'
            conduta = 'Avaliar necessidade de diurético/ultrafiltração e reduzir pressão intratorácica.'
        else:
            categoria = 'Congestão moderada'
            conduta = 'Otimizar balanço hídrico e reavaliar ventilação/PEEP.'
    else:
        categoria = 'Zona cinzenta'
        conduta = 'Integrar com sinais clínicos, lactato e resposta a manobras dinâmicas.'

    return {
        'Categoria': categoria,
        'Sugestão': conduta,
    }


interact_manual(
    avaliar_congestao,
    vci_diam_mm=FloatSlider(value=18, min=8, max=30, step=1, description='VCI (mm)'),
    colapso_percentual=FloatSlider(value=40, min=0, max=100, step=5, description='Colapso (%)'),
    doppler_hepatico=['Normal', 'Reverso/alternante'],
)

## 🧠 Caso Integrado Rápido
**Cenário:** Paciente com choque séptico em noradrenalina 0,3 μg/kg/min, débito urinário baixo e ventilação protetora.

1. Realize ecocardiografia e encontre VTI 14 cm, diâmetro do LVOT 19 mm e colapso da VCI < 10%.
2. Doppler hepático com reversão sistólica.
3. Presença de B-lines difusas.

**Discussão guiada:**
- Débito cardíaco estimado ≈ 3,0 L/min → considerar inotrópico.
- Congestão sistêmica relevante → evitar expansão volêmica, ajustar PEEP.
- Monitorar SvO₂ ou ScvO₂, lactato e repetir POCUS após intervenção.

## 📚 Referências
- Via, G. et al. *International evidence-based recommendations for focused cardiac ultrasound*. Intensive Care Med, 2014.
- Atkinson, P. et al. *Focused ultrasound in the ICU*. Crit Care Clin, 2014.
- ASE. *Focused Cardiac Ultrasound Position Statement*, 2020.
- Guidelines de choque séptico – Surviving Sepsis Campaign 2021.